# Data Science Capstone Project

#### This notebook will be used to contain the contents of my Data Science Capstone Project and the tasks associated with it

In [2]:
import pandas as pd
import numpy as np

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [4]:
#Import libraries for parsing wikipedia page
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [5]:
#Read the html of the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response)

<Response [200]>


In [6]:
#Create a BeautifulSoup html parsing object
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
#Extract the table data frame the page as a list
wiki_page = soup.findAll('td')
for script in soup(["script", "style"]):
    script.extract()    # rip it out

text = soup.get_text()
lines = (line.strip() for line in text.splitlines())
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
text = '\n'.join(chunk for chunk in chunks if chunk)

html_list = text.split('\n')[13:877]

In [8]:
#create a dataframe and fill it with the table data
nbhd_df = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
p_list = []
b_list = []
n_list = []

for x in range(0,288):
    p_list.append(html_list[x*3])
    b_list.append(html_list[x*3+1])
    n_list.append(html_list[x*3+2])
    
nbhd_df['Postcode'] = p_list
nbhd_df['Borough'] = b_list
nbhd_df['Neighbourhood'] = n_list

In [9]:
#remove not assigned boroughs, not assigned neighbourhoods are set equal to borough name
nbhd_df = nbhd_df.loc[nbhd_df['Borough'] != 'Not assigned']
for x in range(len(nbhd_df)):
    if nbhd_df['Neighbourhood'].iloc[x] == 'Not assigned':
        nbhd_df['Neighbourhood'].iloc[x] = nbhd_df['Borough'].iloc[x]

In [10]:
#group and join neighbourhoods by ', '
grouped_df = nbhd_df.groupby(['Postcode','Borough'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ', '.join(x))

In [11]:
#Shape
grouped_df.shape

(103, 3)

In [16]:
#Read in Lats and Longs and assign to appropriate postcode
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')
grouped_df['Latitude'] = ''
grouped_df['Longitude'] = ''
for x in range(len(df_ll)):
    temp_str = df_ll['Postal Code'].iloc[x]
    grouped_df.loc[grouped_df['Postcode'] == temp_str, 'Latitude'] = df_ll['Latitude'].iloc[x]
    grouped_df.loc[grouped_df['Postcode'] == temp_str, 'Longitude'] = df_ll['Longitude'].iloc[x]
    

In [18]:
grouped_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648
